In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [251]:
import pandas as pd
import pickle

In [252]:
# import csv, specific some columns as strings
df = pd.read_csv("../../data/preprocessed_data/subsampled_preprocessed.csv")
#                  dtype={'Pickup_day':object,
#                         'day_of_week': object,
#                        'Pickup_hour':object,
#                        'Pickup_minute':object,
#                        'Pickup_week':object})

In [253]:
attrs1 = ['Season', 'Pickup_month', 'Pickup_week', 'Pickup_day', 'Pickup_wday',
       'Weekend', 'Pickup_hour', 'Time', 'Pickup_minute', 'Zone',
       'Pickup_cell', 'Number_trips']

attrs2 = ['Pickup_week', 'Pickup_wday', 'Pickup_hour', 'Pickup_minute','Pickup_cell', 'Number_trips']

attrs3 = ['Pickup_cell', 'Number_trips']

In [256]:
df2['Pickup_minute'] = df2['Pickup_minute'].apply(lambda x: x//5 * 5)

df2 = df2.groupby(attrs1[:-1]).sum().reset_index()

### Get X and y

In [259]:
from sklearn.preprocessing import OneHotEncoder

In [260]:
attrs = attrs1

df2 = df.copy()[attrs]
_X = df2[attrs[0:len(attrs)-1]]
y = df2["Number_trips"].astype('float')

# One hot encode X,return sparse matrix
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(_X)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [266]:
X = ohe.transform(_X)

In [262]:
with open('../players/model/ohe.pkl', 'wb') as handle:
    pickle.dump(ohe, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [181]:
def get_reg_train(attrs):
    
    df2 = df.copy()[attrs]
    X = df2[attrs[0:len(attrs)-1]]
    y = df2["Number_trips"].astype('float')

    # One hot encode X,return sparse matrix
    ohe = OneHotEncoder()
    X = ohe.fit_transform(X)
    
    return (X, y)

## Model training with cross validation

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
X, y = get_reg_train(attrs)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Multinomial Naive Bayes

In [263]:
from sklearn.naive_bayes import MultinomialNB

In [267]:
# mnb = MultinomialNB()
# mnb.fit(X_train, y_train)

mnb = MultinomialNB()
mnb.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [113]:
mnb.score(X_test, y_test)

0.5640146767072894

In [268]:
with open('../players/model/mnb.pkl', 'wb') as handle:
    pickle.dump(mnb, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [236]:
mnb.predict(test)

array([1.])

### cross validation score
* 1: 0.5569597379648528
* 2: 0.5641991817107989
* 3: 0.5640146767072894

### Linear Regression

In [128]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [157]:
for i in [attrs1, attrs2, attrs3]:
    lr = linear_model.LinearRegression()
    X, y = get_reg_train(i)
    lar.fit(X, y)
    y_pred = lar.predict(X)
    print(i,"=",mean_squared_error(y_pred, y))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [158]:
y_pred = lr.predict(X)
mean_squared_error(y_pred, y)

1.8353711646395263

### mean square error

1. 1.482783895638988
2. 1.8353711646395263
3. 1.5895257371882623

### Lasso Regression

In [128]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [182]:
for i in [attrs1, attrs2, attrs3]:
    lar = linear_model.Lasso(alpha=0.01)
    X, y = get_reg_train(i)
    lar.fit(X, y)
    y_pred = lar.predict(X)
    print(i,"=",mean_squared_error(y_pred, y))

['Season', 'Pickup_month', 'Pickup_week', 'Pickup_day', 'Pickup_wday', 'Weekend', 'Pickup_hour', 'Time', 'Pickup_minute', 'Zone', 'Pickup_cell', 'Number_trips'] = 1.7187530934497663
['Pickup_week', 'Pickup_wday', 'Pickup_hour', 'Pickup_minute', 'Pickup_cell', 'Number_trips'] = 1.7848726756374265
['Pickup_cell', 'Number_trips'] = 1.836361042669515


In [183]:
for i in [attrs1, attrs2, attrs3]:
    lar = linear_model.Lasso(alpha=0.1)
    X, y = get_reg_train(i)
    lar.fit(X, y)
    y_pred = lar.predict(X)
    print(i,"=",mean_squared_error(y_pred, y))

['Season', 'Pickup_month', 'Pickup_week', 'Pickup_day', 'Pickup_wday', 'Weekend', 'Pickup_hour', 'Time', 'Pickup_minute', 'Zone', 'Pickup_cell', 'Number_trips'] = 1.9154787853793123
['Pickup_week', 'Pickup_wday', 'Pickup_hour', 'Pickup_minute', 'Pickup_cell', 'Number_trips'] = 1.9154787853793123
['Pickup_cell', 'Number_trips'] = 1.9154787853793123


### Poisson regression

In [176]:
# Poisson regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [9]:
formula = 'Number_trips~ Pickup_week+Pickup_wday+Pickup_hour+Pickup_minute+Pickup_cell

data = df2

In [10]:
pois_glm = smf.glm(formula=formula, data=data, family=sm.families.Poisson()).fit()

MemoryError: 